In [1]:
map_relations = {'Comment':0, 'Contrast':1, 'Correction':2, 'Question-answer_pair':3, 'Acknowledgement':4,'Elaboration':5,
                 'Clarification_question':6, 'Conditional':7, 'Continuation':8, 'Result':9, 'Explanation':10, 'Q-Elab':11,
                 'Alternation':12, 'Narration':13, 'Confirmation_question':14, 'Sequence':15, 'Break':16}

In [2]:
reverse_relations = {0:'Comment', 1:'Contrast', 2:'Correction', 3:'Question-answer_pair', 4:'Acknowledgement',5:'Elaboration',
                 6:'Clarification_question', 7:'Conditional', 8:'Continuation', 9:'Result', 10:'Explanation', 11:'Q-Elab',
                 12:'Alternation', 13:'Narration', 14:'Confirmation_question', 15:'Sequence', 16:'Break'}

In [3]:
import pickle
import json
from collections import Counter, defaultdict 
from sklearn.metrics import precision_recall_fscore_support,classification_report, ConfusionMatrixDisplay, confusion_matrix

In [15]:
home = %pwd
gold_path = home + '/data/TEST_102_bert.json'
preds_path = home + '/pickles/scores_multi_d10.pkl'

In [23]:
with open(gold_path, 'r') as jf:
    test_gold = json.load(jf)

In [16]:
with open(preds_path, 'rb') as f:
    test_pred = pickle.load(f)

In [24]:
len(test_pred), len(test_gold)

(5338, 102)

combine test pred and test gold info: <br>
get speaker, global turn info from gold, and incoming result and continuation info from preds...

In [18]:
test_pred[:4]

[[0, 0, 1, 0, 4], [0, 0, 2, 8, 8], [0, 2, 3, 5, 2], [0, 3, 4, 9, 9]]

In [25]:
for i, game in enumerate(test_gold):
    preds = [[e[1], e[2], e[4]] for e in test_pred if e[0]==i]
    pred_rels = []
    for p in preds:
        pred_rel = {}
        pred_rel['x'] = p[0]
        pred_rel['y'] = p[1]
        pred_rel['type'] = reverse_relations[p[2]]
        pred_rels.append(pred_rel)
    game['predicted_relations'] = pred_rels

In [28]:
#test_gold[0]['predicted_relations']

step one <br>
adds info to edus that is necessary for more second pass (Narration) processing.
info: global turn, architect edu index in turn, 1/0 result incoming, edu type
returns only narration relations. 

In [60]:
def is_nl(edu):
    """
    if every word in alphanumeric
    """
    nl = 1
    words = edu.split(' ')
    # print(words)
    for word in [w for w in words if w != '']:
        if not contains_number(word) or len(word)<5:
            nl = 0
            break
    return nl

def contains_number(string):
    return any(char.isdigit() for char in string)

In [81]:
for game in test_gold:
    # rels = game['predicted_relations']
    rels = game['relations']
    edus = game['edus']
    new_rels = []

    #add turn index for arch and global turn info to all edus
    ind_cnt = 0
    global_cnt = 0
    last_speaker = None
    global_index = 0
    for edu in edus:
        edu['global_index'] = global_index
        global_index += 1
        speaker = edu['speaker']
        if speaker == last_speaker:
            edu['turn'] = global_cnt
        else:
            last_speaker = speaker
            global_cnt += 1
            edu['turn'] = global_cnt
        if speaker == 'Architect':
            edu['turn_ind'] = ind_cnt
            ind_cnt += 1
            edu['type'] = 0
        elif speaker == 'Builder':
            ind_cnt = 0
            #also add type info
            if is_nl(edu['text']):
                edu['type'] = 1 
            else:
                edu['type'] = 0
        #add field for incoming result information
        edu['res'] = 0
        edu['nar_start'] = 0

    #add incoming Result information
    for rel in rels:
        ind = rel['y']
        if rel['type'] == 'Result':
            edus[ind]['res'] = 1
        # if rel['type'] == 'Acknowledgement':
        #     edus[ind]['ack'] = 1
        #add first continuation!!!
        if rel['x'] == 0 and rel['type'] == 'Continuation':
            edus[ind]['nar_start'] = 1

        #keep only specified type of relations
#         if rel['type'] == 'Narration':
#             new_rels.append(rel)         
#     game['relations'] = new_rels

In [82]:
predictions = []
# for game in dev_32:
for game in test_gold:
        narration_guesses = []
        actions_happening = 0
        actions_happened = 0
        narr = [0,0]
        try:
            start = [edu['global_index'] for edu in game['edus'] if edu['nar_start'] == 1][0]
            narr[0] = start
        except IndexError:
            print('no start')
            pass   
        for edu in game['edus'][start+1:]:
    #         print(edu['type'],actions_happening, actions_happened)
            if edu['type'] == 1 and actions_happening == 0: #if builder is moving
                actions_happening = 1
            if edu['type'] == 0  and actions_happening == 1: #if someone is talking now
                actions_happened = 1
            # if edu['speaker'] == 'Architect' and edu['res'] == 1 and actions_happened == 1:
            #     narr[1] = edu['global_index']
            if edu['speaker'] == 'Architect' and edu['res'] == 1:
                narr[1] = edu['global_index']

                narration_guesses.append(tuple(narr))
                narr = [0,0]
                narr[0] = edu['global_index']
                actions_happening = 0
                actions_happened = 1
        predictions.append(narration_guesses)


no start


In [83]:
len(predictions)

102

In [74]:
predictions[0]

[(2, 6), (6, 13)]

In [84]:
#format gold
gold = []
for game in test_gold:
    narrs = []
    for rel in game['relations']:
        if rel['type'] == 'Narration':
            narrs.append((rel['x'], rel['y']))
    gold.append(narrs)
        

In [76]:
len(gold)

102

In [41]:
gold[0]

[(2, 6), (6, 13), (13, 18)]

In [85]:
#now rearrange
gold_narrations = []
pred_narrations = []

for i, game in enumerate(gold):
    preds = predictions[i]
    for rel in game:
        if rel in preds:
            gold_narrations.append(1)
            pred_narrations.append(1)
        if rel not in preds:
            gold_narrations.append(1)
            pred_narrations.append(0)
    for rel in preds:
        if rel not in game:
            gold_narrations.append(0)
            pred_narrations.append(1)
            

In [86]:
len(gold_narrations), len(pred_narrations)

(785, 785)

In [87]:
scores = precision_recall_fscore_support(gold_narrations, pred_narrations, average='binary')

In [80]:
#using predicted relations 
scores

(0.6873111782477341, 0.603448275862069, 0.6426553672316384, None)

In [59]:
#using gold relations 
scores

(0.9504814305364512, 0.916445623342175, 0.9331532748143146, None)

In [88]:
scores

(0.9577080491132333, 0.9310344827586207, 0.9441829186281103, None)